In [ ]:
'''
- このプログラムについて
KMLファイルを一定間隔の座標群CSV形式で出力する．

入力：KMLファイル
    Googleマイマップでルートを引くとダウンロードできる
    ファイル名は「速度(km/h)_分かりやすい名前」推奨
        例：「80_**********************************」

出力：CSVファイル
    Latitude,Longitude
    間隔は「指定した速度*1000/3600」m

- 使い方
1. このプログラムと同じディレクトリに，作業フォルダを作成し，作業フォルダ名を"FOLDER_PATH"に代入する．
2. 作業フォルダ内に下記のディレクトリを作成する．
    - CSV_DrivingLog
    - CSV_DrivingLog_inserted
    - CSV_FoliumMap
    - KML_DrivingRoute
    - KML_DrivingRoute_inserted
    - CSV_**************table
    - CSV_**************_EnergyMap
3. KMLファイルを用意し，ファイル名の先頭で速度を指定する．
4. KML_DrivingRouteフォルダにKMLファイルを入れ，本プログラムを実行すると，CSVフォルダに出力し，KMLファイルはKML_insertedフォルダに移動する．

'''

# module
import os
import re
from pathlib import Path
import webbrowser
from os import path
import json
import time
import shutil
import csv
import math
import pandas as pd
import folium
from ConsumedEnergyCalculator import self_execution
import datetime
import sklearn
from sklearn.neighbors import KNeighborsClassifier
import pyodbc

In [ ]:
now_jst = datetime.datetime.now()
now_jst_result = str(now_jst.strftime('%Y%m%d-%H%M%S'))
#FOLDER_PATH = "\\{}".format(now_jst_result)  # 作業フォルダ
FOLDER_PATH = ""  # 作業フォルダ
KML_PATH = rf"\KML_DrivingRoute"  # 入力フォルダ
KML_inserted_PATH = rf"\KML_DrivingRoute_inserted"  # 入力済みフォルダ
CSV_PATH = rf"\CSV_DrivingLog"  # 出力フォルダ
CSV_FOLIUM_PATH = r"\CSV_FoliumMap"  # CSV の folium マップ出力フォルダ
#ACCESS_NUMBER = 5  # いつも**************アクセス数（1アクセスにつき100入力まで）
#CONSTANT_SPEED = 100 # 定速速度はファイル名から自動で判別するようにする（この値はデフォルト値）

# ヒュベニの公式
# https://butter-tiger.hatenablog.com/entry/2020/08/20/222650
POLE_RADIUS = 6356752  # 極半径(短半径)
EQUATOR_RADIUS = 6378137  # 赤道半径(長半径)
E = 0.081819191042815790  # 離心率
E2 = 0.006694380022900788  # 離心率の２乗

def distance(_lat1, _long1, _lat2, _long2):
    lat1 = math.radians(_lat1)
    long1 = math.radians(_long1)
    lat2 = math.radians(_lat2)
    long2 = math.radians(_long2)
    m_lat = (lat1 + lat2) / 2  # 平均緯度
    d_lat = abs(lat1 - lat2)  # 緯度差
    d_lon = abs(long1 - long2)  # 経度差
    W = math.sqrt(1 - E2 * math.pow(math.sin(m_lat), 2))
    M = EQUATOR_RADIUS * (1 - E2) / math.pow(W, 3)  # 子午線曲率半径
    N = EQUATOR_RADIUS / W  # 卯酉線曲率半径
    # d = math.sqrt(math.pow(M*d_lat,2) + math.pow(N*d_lon*math.cos(m_lat),2) + math.pow(point_a.altitude-point_b.altitude,2))
    d = math.sqrt(math.pow(M * d_lat, 2) + math.pow(N * d_lon * math.cos(m_lat), 2))
    return d


def search_filelist():
    
    p = Path(rf"{os.getcwd()}{KML_PATH}")
    print(p)

    filenames = []

    for file in p.iterdir():
        if file.is_dir():
            continue

        # pathstrings = str(file).split('/')
        # filename = file.name
        # print(pathstrings)
        if re.match(".+" + r".kml", file.name):
            print(file.name, ":KML file")
            filenames.append(
                rf"{os.getcwd()}{KML_PATH}\{file.name}"
            )
        else:
            print(file.name, ":not KML file:")
        # filenames.append(file.name)
    return filenames


# KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返すメソッド
def readLinksKML(filename):
    lineSirungTab1 = r"<LineString>"
    lineSirungTab2 = r"</LineString>"
    isLineString = 0
    f = open(filename, "r", encoding="UTF-8")
    datalist = f.readlines()
    GPSlist = [["Latitude", "Longitude"]]
    for data in datalist:
        if isLineString == 1:
            if lineSirungTab2 in data:
                # print('ここまでが対象データ')
                isLineString = 0
            else:
                target_data = data.replace(" ", "").replace(r",0", "")
                if re.match(r"[0-9]+.?[0-9]*,[0-9]+.?[0-9]*", target_data):
                    target_data2 = re.split(",", target_data)
                    appendList = [float(target_data2[1]), float(target_data2[0])]
                    GPSlist.append(appendList)
                    # print(appendList)
        elif isLineString == 0:
            if lineSirungTab1 in data:
                # print('ここからが対象データ')
                isLineString = 1
        else:
            print("よきせぬれーがい")
    GPSlist.remove(["Latitude", "Longitude"])
    return GPSlist


def calcDistance(list_GPS):
    list_lat_long_dist = [
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    ]
    before_lat = 0
    before_long = 0
    dist_sum = 0
    for row in list_GPS:
        if before_lat > 0:
            dist = distance(before_lat, before_long, row[0], row[1])
            appendList = [before_lat, before_long, row[0], row[1], dist]
            list_lat_long_dist.append(appendList)
            dist_sum = dist_sum + dist
            # print(appendList)
        before_lat = row[0]
        before_long = row[1]
    list_lat_long_dist.remove(
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    )
    return list_lat_long_dist, dist_sum


def normalizedCoordinatesGenerator(list_lat_long_dist, normalized_dist):
    normalizedList = [["Latitude", "Longitude"]]
    rest = normalized_dist
    _lat = list_lat_long_dist[0][0]
    _long = list_lat_long_dist[0][1]
    appendList = [_lat, _long]
    normalizedList.append(appendList)
    coordinatesNumber = 1

    for row in list_lat_long_dist:
        row_loop = 0
        if rest > row[4]:
            rest = rest - row[4]
            # print('rest:', str(rest))
        else:
            while rest + normalized_dist * row_loop < row[4]:
                _lat = (row[2] - row[0]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[0]
                _long = (row[3] - row[1]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[1]
                appendList = [_lat, _long]
                normalizedList.append(appendList)
                coordinatesNumber = coordinatesNumber + 1
                row_loop = row_loop + 1
                # print('rest:', str(rest))
            rest = rest - row[4] + normalized_dist * row_loop
    
    return normalizedList, coordinatesNumber

coordinatesNumber = 0.00
# MAIN STREAM
def GPSListGenerator(filename):
    print("try generate {}.".format(filename))
    list_GPS = readLinksKML(filename)  # KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返す
    # print(list_GPS)    #(lat, long)
    list_lat_long_dist, dist_sum = calcDistance(list_GPS)
    # print(list_lat_long_dist)  #(lat1, long1, lat2, long2, dist)

    NORMALIZED_DISTANCE = 10000
    __filename = filename.split('\\')
    #print(__filename[-1])
    if re.match("[0-9]+" + r"_" + ".+" + r".kml", __filename[-1]):
        filename_split = __filename[-1].split('_')
        CONSTANCE_SPEED = int(filename_split[0])
        print(filename, "  speed:", CONSTANCE_SPEED)
        NORMALIZED_DISTANCE = CONSTANCE_SPEED * 1000 / 3600
    else:
        print("ファイル名で速度を指定してください")

    normalized_dist = NORMALIZED_DISTANCE
    print(
        "distance:",
        str(round((dist_sum / 1000), 3)),
        "km,  normalized_dist:",
        str(normalized_dist),
        "m",
    )
    list_normalized_lat_long, coordinatesNumber = normalizedCoordinatesGenerator(
        list_lat_long_dist, normalized_dist
    )
#     print('now printing')
#     print(list_normalized_lat_long)
    return list_normalized_lat_long

#上のGPSListGeneratorのoutputをcsvファイルとして出力
def output_lat_long(filename, list_normalized_lat_long):
    filename_csv = filename.replace(KML_PATH, CSV_PATH).replace(r'.kml', r'.csv')
    print("output file:", filename_csv, "    座標数:", str(coordinatesNumber), "\n")
    # 2次元配列→CSV変換:https://rikei-danshi.work/entry/python-2darray-csv
    with open(filename_csv, "w") as file:
        writer = csv.writer(file, lineterminator="\n")
        writer.writerows(list_normalized_lat_long)


def move_file(filename):
    filename_kml_inserted = filename.replace(KML_PATH, KML_inserted_PATH)
    shutil.move(filename, filename_kml_inserted)


def display_folium_map_from_csv(file_path):
    output_folium_file_path = rf"{path.dirname(__file__)}{FOLDER_PATH}{CSV_FOLIUM_PATH}\{os.path.basename(file_path).split('.')[0]}.html"
    df_output_points = pd.read_csv(
        rf"{path.dirname(__file__)}{FOLDER_PATH}{CSV_PATH}\{os.path.basename(file_path).split('.')[0]}.csv"
    )
    folium_map = folium.Map(
        location=[
            df_output_points.at[len(df_output_points) // 2, "Latitude"],
            df_output_points.at[len(df_output_points) // 2, "Longitude"],
        ],
        zoom_start=13,
    )
    for index, point in df_output_points.iterrows():
        folium.Marker(location=[point["Latitude"], point["Longitude"]]).add_to(
            folium_map
        )
    folium_map.save(output_folium_file_path)
    webbrowser.open(output_folium_file_path, new=2)

In [ ]:
df_SQL_data = pd.DataFrame({'LINK_ID':[],'Latitude':[],'Longitude':[]})
def get_LINKS_from_SQL_SERVER_not_hokkaido(LATITUDE_max,LATITUDE_min,LONGITUDE_max,LONGITUDE_min):
    driver='{SQL Server}'
    server = '****************************'
    database = '****************************'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    cursor.execute( "SELECT LINK_ID ,LATITUDE ,LONGITUDE FROM [****************************].[dbo].[LINKS] where (LATITUDE < ? and LATITUDE > ?) and (LONGITUDE <? and LONGITUDE> ?) order by LINK_ID", LATITUDE_max,LATITUDE_min,LONGITUDE_max,LONGITUDE_min) 
    rows = cursor.fetchall()
    
    cursor.close()
    connect.close()
    return rows

def get_LINKS_from_SQL_SERVER_hokkaido(LATITUDE_max,LATITUDE_min,LONGITUDE_max,LONGITUDE_min):
    driver='{SQL Server}'
    server = '****************************'
    database = '****************************'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    cursor.execute( "SELECT LINK_ID ,LATITUDE ,LONGITUDE FROM [****************************].[dbo].[LINKS_GSI20] where (LATITUDE < ? and LATITUDE > ?) and (LONGITUDE <? and LONGITUDE> ?) order by LINK_ID", LATITUDE_max,LATITUDE_min,LONGITUDE_max,LONGITUDE_min) 
    rows = cursor.fetchall()
    
    cursor.close()
    connect.close()
    return rows

def get_Semantic_id():
    driver='{SQL Server}'
    server = '****************************'
    database = '****************************'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    cursor.execute( "select max(SEMANTIC_LINK_ID) FROM [****************************].[dbo].[SEMANTIC_LINKS] where SEMANTIC_LINK_ID < 995" ) 
    rows = cursor.fetchall()
    
    cursor.close()
    connect.close()
    return rows

def table_name():
    return "*****************************"
def column_list():
    return "(SEMANTIC_LINK_ID, DRIVER_ID, LINK_ID, SEMANTICS)"

def inser_data_to_SEMANTIC_LINKS(data):
    driver='{SQL Server}'
    server = '****************************'
    database = '****************************'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    for i in range(len(data)):
        list1 = data.loc[i].to_list()
        stmt = """
        INSERT INTO {} {} 
        VALUES {}
        """.format(table_name(),column_list(),tuple(list1))
        try:
            cursor.execute(stmt)
            cursor.commit()
        except pyodbc.IntegrityError as err:
                        # 主キー違反の場合には読み飛ばす
                            continue
        except Exception as e:
                            print("---")
                            print(e)
                            print("HINT: トリップの最初のレコードだけエラーが出る様子.")
                            print("Excecuted SQL below.")
                            print(stmt)
                            print("---")
    cursor.close()
    connect.close()

In [ ]:
if __name__ == "__main__":
    for filename in search_filelist():
        #read kml and convert it to a DataFrame
        df_kml = pd.DataFrame(GPSListGenerator(filename)[1:],columns =GPSListGenerator(filename)[0])
        #For saving RAM,so we query SQL Server for the range only we need
        LATITUDE_max = df_kml.Latitude.max()+0.000000001
        LATITUDE_min = df_kml.Latitude.min()-0.000000001
        LONGITUDE_max = df_kml.Longitude.max()+0.000000001
        LONGITUDE_min = df_kml.Longitude.min()-0.000000001
        #check if query is in Hokkaido
        if LATITUDE_min > 41.1614271:    #LATITUDE of 佐井村
            datas = get_LINKS_from_SQL_SERVER_hokkaido(LATITUDE_max,LATITUDE_min,LONGITUDE_max,LONGITUDE_min)
        else:
            datas =get_LINKS_from_SQL_SERVER_not_hokkaido(LATITUDE_max,LATITUDE_min,LONGITUDE_max,LONGITUDE_min)
        #insert data to df_SQL_data(DataFrame)
        for i in range(len(datas)):
            df_SQL_data.loc[i] = list(datas[i])
        #To match link_id of roads, a Knn classifier is trained with datas extracted from the SQL Server
        train_data = df_SQL_data.iloc[:,1:3]
        train_label = df_SQL_data.iloc[:,0]
        Classifier = KNeighborsClassifier(n_neighbors=1)
        Classifier.fit(train_data,train_label)
        #Use trained classifier to match roads
        #you can trust this classifier cause of I set n_neighbors to 1,and it must overfitting
        #so its accuracy will be 100%
        output = Classifier.predict(df_kml.iloc[:,:])
        output_for_inserting = pd.Series(output).unique()
        #The following DataFrame was created to aggregate the training results and to organize the data
        Data_for_inserting = pd.DataFrame({'SEMANTIC_LINK_ID':[],
                                   'DRIVER_ID':[],
                                   'LINK_ID':[],
                                   'SEMANTICS':[]})
        #SEMANTIC_LINK_ID is dependent on the maximum non-test id in the database
        SEMANTIC_LINK_ID = int(list(get_Semantic_id()[0])[0])+1
        #SEMANTICS is dependent on the kml filename
        SEMANTICS = filename.split('\\')[-1].split('.')[0].split('_')[-1]
        Data_for_inserting.LINK_ID = output_for_inserting  
        Data_for_inserting.SEMANTIC_LINK_ID = SEMANTIC_LINK_ID
        #you need to set a DRIVER_ID
        print('セマンティック'+SEMANTICS+'のDRIVER_IDを入力（指定）してください:')
        DRIVER_ID = input()
        DRIVER_ID = int(DRIVER_ID)
        Data_for_inserting.DRIVER_ID = DRIVER_ID
        Data_for_inserting.SEMANTICS = SEMANTICS
        #insert aggregated datas to SQL server
        inser_data_to_SEMANTIC_LINKS(Data_for_inserting)
        #save coordinate information for some other needs
        output_lat_long(filename,GPSListGenerator(filename))
        display_folium_map_from_csv(filename)
        move_file(filename)